In [1]:
# Nome: Francisco Mateus dos Anjos Silva
# Matrícula: 521783
# Disciplina: Introdução à Ciência de Dados

# Extraindo dados da internet (Web Scraping) com scrapy

## Construindo dataset sobre Notícias com dados do G1

###  Etapas:
#### 1 - Extrair dados da internet (Web Scraping)
#### 2 - Construir dataset com os dados extraídos da internet
#### 3 - Análise exploratória dos dados
#### 4 - Tratamento / limpeza dos dados
#### 5 - Exportar dataframe do pandas para CSV

In [2]:
# Importando as bibliotecas

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## 1 - Extraindo dados da internet (Web Scraping)

In [4]:
# Vamos extrair dados do site do G1 (notícias): 
# https://g1.globo.com/

### Configurando Ambiente

In [5]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
# Exibir versão do Python
import platform
platform.python_version()

try: # Checando se Scrapy está instalado
    import scrapy
except:
    !pip install scrapy
    import scrapy
from scrapy.crawler import CrawlerProcess
from scrapy.utils.project import get_project_settings

In [6]:
import json

class JsonWriterPipeline(object):

    # Função para gerar/abrir arquivo JSON
    def open_spider(self, spider):
        self.file = open('g1result.jl', 'w')

    # Fechar arquivo após escrita
    def close_spider(self, spider):
        self.file.close()

    # Inserir itens coletados da página WEB no arquivo JSON criado
    def process_item(self, item, spider):
        line = json.dumps(dict(item)) + "\n"
        self.file.write(line)
        return item

In [7]:
import logging

class G1Spider(scrapy.Spider):
    name = "g1"
    
    categories = ['economia/agronegocios/', 
                  'economia/agronegocios/agro-a-industria-riqueza-do-brasil/',
                  'ciencia/',
                  'bemestar/viva-voce/',
                  'rj/rio-de-janeiro/carnaval/2022/',
                  'sp/sao-paulo/carnaval/2022/',
                  'economia/',
                  'economia/bitcoin/',
                  'educacao/',
                  'educacao/enem/2021/',
                  'empreendedorismo/',
                  'empreendedorismo/pegn/',
                  'fato-ou-fake/',
                  'especiais/guia-de-compras/',
                  'inovacao/',
                  'loterias/',
                  'meio-ambiente/',
                  'monitor-da-violencia/',
                  'mundo/',
                  'mundo/ucrania-russia/',
                  'olha-que-legal/',
                  'politica/',
                  'pop-arte/cinema/',
                  'pop-arte/games/',
                  'saude/',
                  'tecnologia/',
                  'trabalho-e-carreira/',
                  'turismo-e-viagem/']
    
    urls = []
    for category in categories:
        url = 'https://g1.globo.com/' + str(category)
        urls.append(url)
    
    # Lista de Strings (URLs)   
    start_urls = urls
    
    # Configuração obrigatória de pipeline para geração de arquivo de saída
    custom_settings = {
        'LOG_LEVEL': logging.WARNING,
        'ITEM_PIPELINES': {'__main__.JsonWriterPipeline': 1}, 
        'FEED_FORMAT':'json',                                 
        'FEED_URI': 'g1result.json'                        
    }
    
    # Parse da página principal a ser crawleada
    def parse(self, response):
                            
        for news in response.css('.bastian-page .bastian-feed-item'):
            
            title = news.css('.feed-post-link::text').extract_first()
            description = news.css('.feed-post-body-resumo::text').extract_first()
            image_url = news.css('.bstn-fd-picture-image::attr(src)').extract_first()
            link = news.css('.feed-post-link::attr(href)').extract_first()
            categoria = news.css('.feed-post-metadata-section::text').extract_first()
            
            yield{
                'titulo': title,
                'descricao': description,
                'url_imagem': image_url,
                'link': link,
                'categoria': categoria
            }
        

## 2 - Construindo dataset (gerando arquivo) com os dados extraídos da internet

In [8]:
process = CrawlerProcess(get_project_settings())

# Iniciando processo
process.crawl(G1Spider)
process.start()

2022-05-19 11:10:46 [scrapy.utils.log] INFO: Scrapy 2.6.1 started (bot: scrapybot)
2022-05-19 11:10:46 [scrapy.utils.log] INFO: Versions: lxml 4.6.3.0, libxml2 2.9.12, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 22.4.0, Python 3.9.7 (default, Sep 16 2021, 13:09:58) - [GCC 7.5.0], pyOpenSSL 21.0.0 (OpenSSL 1.1.1l  24 Aug 2021), cryptography 3.4.8, Platform Linux-5.13.0-41-generic-x86_64-with-glibc2.31
2022-05-19 11:10:46 [scrapy.crawler] INFO: Overridden settings:
{'LOG_LEVEL': 30}
2022-05-19 11:10:46 [py.warnings] WARNING: /home/mateus/anaconda3/envs/env-introducao-ciencia-dados/lib/python3.9/site-packages/scrapy/extensions/feedexport.py:289: ScrapyDeprecationWarning: The `FEED_URI` and `FEED_FORMAT` settings have been deprecated in favor of the `FEEDS` setting. Please see the `FEEDS` setting docs for more details
  exporter = cls(crawler)



### Carregando arquivo criado com os dados extraídos

In [9]:
import pandas as pd
# Carregando JSON criado para visualizar saída
df = pd.read_json('g1result.jl', lines=True)
df.head()

,titulo,descricao,url_imagem,link,categoria
0,Ibovespa passa a subir ajudada por ações da El...,"Na quarta-feira, o principal índice de ações d...",https://s2.glbimg.com/NyAWUEYBa2bzlCJq3FgrS_9m...,https://g1.globo.com/economia/noticia/2022/05/...,Economia
1,Agricultor tem opção de picape em que pode con...,A Nova Nissan Frontier possui força e estilo p...,https://s2.glbimg.com/iOJ6--Ov9NfepV8queDkwTio...,https://g1.globo.com/economia/agronegocios/agr...,NISSAN
2,Dólar retoma trajetória de queda nesta quinta,"Na quarta-feira, a moeda norte-americana subiu...",None,https://g1.globo.com/economia/noticia/2022/05/...,Economia
3,Justiça libera saque de FGTS a família com dep...,Tribunal autorizou uso de FGTS para doença não...,https://s2.glbimg.com/4Ce3Jq73qtrpF9wMIdjji1QG...,https://g1.globo.com/economia/noticia/2022/05/...,Economia
4,Dinheiro na poupança 'encolhe' há 20 meses seg...,"Em abril, rentabilidade no acumulado em 12 mes...",https://s2.glbimg.com/TJHYXXEv4P3W7l_FOUTHtkLk...,https://g1.globo.com/economia/noticia/2022/05/...,Economia


## 3 - Análise exploratória dos dados

In [10]:
print("Número de instâncias: " + str(df.shape[0]))
print("Número de atributos: " + str(df.shape[1]))

Número de instâncias: 258
Número de atributos: 5


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 258 entries, 0 to 257
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   titulo      258 non-null    object
 1   descricao   248 non-null    object
 2   url_imagem  245 non-null    object
 3   link        256 non-null    object
 4   categoria   256 non-null    object
dtypes: object(5)
memory usage: 10.2+ KB


In [12]:
# Quantidade de valores nulos por atributo
df.isna().sum()

titulo         0
descricao     10
url_imagem    13
link           2
categoria      2
dtype: int64

## 4 - Tratamento / limpeza dos dados

In [13]:
# Excluindo linhas que têm algum atributo nulo
df.dropna(inplace=True)
df.shape

(240, 5)

In [14]:
# Quantidade de valores nulos por atributo
df.isna().sum()

titulo        0
descricao     0
url_imagem    0
link          0
categoria     0
dtype: int64

In [15]:
# Quantidade de instâncias por categoria
df['categoria'].value_counts()

 Economia                 11
 Guia de Compras          10
 Loterias                 10
 Ucrânia e Rússia          9
 Meio Ambiente             9
                          ..
 Amapá                     1
 Monitor da Violência      1
 Viva Você                 1
 Rio Grande do Norte       1
 G1 Turismo e Viagem       1
Name: categoria, Length: 68, dtype: int64

### 5 - Exportando dataframe do pandas para CSV

In [20]:
df.to_csv('g1result.csv', index=False, encoding='utf-8')